<a href="https://colab.research.google.com/github/Amisha1019/Emotion-Detection-Internship-/blob/main/Task_5_emotion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
code = r"""
# --- Paste the full content of sign_language_detector_app.py here ---
# (Copy the entire code I gave earlier and paste inside this string)
"""

with open("sign_language_detector_app.py", "w") as f:
    f.write(code)

print("File created successfully!")


File created successfully!


In [29]:
!ls -l


total 8
drwxr-xr-x 1 root root 4096 Nov 17 14:29 sample_data
-rw-r--r-- 1 root root  141 Nov 19 17:24 sign_language_detector_app.py


In [31]:
from google.colab import files
files.download("sign_language_detector_app.py")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [54]:
!streamlit run sign_language_detector_app.py &>/dev/null&

In [55]:
!pip install pyngrok
from pyngrok import ngrok


In [56]:
"""
Sign Language Detector - Streamlit App (single-file)


Features:
- Real-time webcam sign recognition (using MediaPipe hand landmarks + a trained Keras model)
- Image upload recognition
- Time-window operation: app only performs predictions between 18:00 and 22:00 Asia/Kolkata (6 PM - 10 PM)
- Logging of detections (one CSV for image uploads, one CSV for webcam/video detections) with download buttons
- Training utilities (data preprocessing helpers + example model architecture). You'll need to train and save `sign_model.h5` before full operation.


How to run:
1. Create a virtualenv and install requirements:
pip install streamlit mediapipe opencv-python tensorflow pandas numpy scikit-learn python-dateutil


2. Run:
streamlit run sign_language_detector_app.py


Notes:
- This is a single-file app that contains both GUI and helper functions. Replace/modify the training section as needed for your dataset.
- For best results for video sequences, collect sequences of hand landmarks (e.g., 30 frames) for each sign and train an LSTM/GRU on sequences. The example below includes both an image-based (frame-level) model and a sequence model outline.


"""

"\nSign Language Detector - Streamlit App (single-file)\n\n\nFeatures:\n- Real-time webcam sign recognition (using MediaPipe hand landmarks + a trained Keras model)\n- Image upload recognition\n- Time-window operation: app only performs predictions between 18:00 and 22:00 Asia/Kolkata (6 PM - 10 PM)\n- Logging of detections (one CSV for image uploads, one CSV for webcam/video detections) with download buttons\n- Training utilities (data preprocessing helpers + example model architecture). You'll need to train and save `sign_model.h5` before full operation.\n\n\nHow to run:\n1. Create a virtualenv and install requirements:\npip install streamlit mediapipe opencv-python tensorflow pandas numpy scikit-learn python-dateutil\n\n\n2. Run:\nstreamlit run sign_language_detector_app.py\n\n\nNotes:\n- This is a single-file app that contains both GUI and helper functions. Replace/modify the training section as needed for your dataset.\n- For best results for video sequences, collect sequences of 

In [57]:
!pip install streamlit

In [58]:
!pip install streamlit mediapipe opencv-python tensorflow pandas numpy scikit-learn python-dateutil pytz

In [59]:
import streamlit as st
import cv2
import numpy as np
import mediapipe as mp
import pandas as pd
import io
import os
from datetime import datetime, time
import pytz
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, BatchNormalization, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [60]:
!pip install mediapipe opencv-python tensorflow pandas numpy scikit-learn python-dateutil

In [61]:
TIMEZONE = 'Asia/Kolkata'
OPERATION_START = time(18, 0) # 18:00
OPERATION_END = time(22, 0) # 22:00
MODEL_PATH = 'sign_model.h5' # Put your trained model here
SEQUENCE_LENGTH = 30 # For video sequence model (frames)


# Labels used by the model (example - change to your dataset labels)
LABELS = ['hello', 'thank_you', 'yes', 'no', 'iloveu']


# -----------------------------
# Utilities
# -----------------------------
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

In [62]:
def in_operation_hours(now=None):
    tz = pytz.timezone(TIMEZONE)
    if now is None:
        now = datetime.now(tz)
    local_time = now.time()
    if OPERATION_START <= OPERATION_END:
        return OPERATION_START <= local_time <= OPERATION_END
    else:
        # Overnight window (not used here, but general)
        return local_time >= OPERATION_START or local_time <= OPERATION_END

In [63]:
def hands_to_feature_vector(hand_landmarks):
    """Convert MediaPipe hand landmarks to a flat feature vector (normalized).
    Expects one hand landmarks object. Returns 63-dim vector (21 x (x,y,z))."""
    coords = []
    for lm in hand_landmarks.landmark:
        coords.extend([lm.x, lm.y, lm.z])
    coords = np.array(coords)
    # normalize by subtracting wrist coordinates (first landmark)
    wrist_x, wrist_y, wrist_z = coords[0], coords[1], coords[2]
    coords[0::3] -= wrist_x
    coords[1::3] -= wrist_y
    coords[2::3] -= wrist_z
    return coords

In [64]:
def predict_frame_model(model, image):
    """Detect hands, extract features, and predict using a frame-level model."""
    with mp_hands.Hands(static_image_mode=True, max_num_hands=1) as hands:
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image_rgb)
        if not results.multi_hand_landmarks:
            return None, None
        hand = results.multi_hand_landmarks[0]
        fv = hands_to_feature_vector(hand)
        fv = fv.reshape(1, -1)
        pred = model.predict(fv)
        label_idx = np.argmax(pred)
        return LABELS[label_idx], pred[0]

In [65]:
def build_frame_model(num_classes=len(LABELS)):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(63,)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [66]:
def build_sequence_model(num_classes=len(LABELS), seq_len=SEQUENCE_LENGTH):
    model = Sequential()
    model.add(LSTM(64, return_sequences=True, input_shape=(seq_len, 63)))
    model.add(Dropout(0.3))
    model.add(LSTM(64))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [67]:
image_log_columns = ['timestamp', 'label', 'confidence', 'filename']
video_log_columns = ['timestamp', 'label', 'confidence', 'frame_index']

In [68]:
if 'image_log' not in st.session_state:
    st.session_state['image_log'] = pd.DataFrame(columns=image_log_columns)
if 'video_log' not in st.session_state:
    st.session_state['video_log'] = pd.DataFrame(columns=video_log_columns)

2025-11-19 17:37:10.414 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 17:37:10.417 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [69]:
def log_image_detection(label, conf, filename):
    df = st.session_state['image_log']
    df = df.append({
        'timestamp': datetime.now(pytz.timezone(TIMEZONE)).isoformat(),
        'label': label,
        'confidence': float(conf),
        'filename': filename
    }, ignore_index=True)
    st.session_state['image_log'] = df

In [70]:
def log_video_detection(label, conf, frame_idx):
    df = st.session_state['video_log']
    df = df.append({'timestamp': datetime.now(pytz.timezone(TIMEZONE)).isoformat(),
    'label': label, 'confidence': float(conf), 'frame_index': int(frame_idx)}, ignore_index=True)
    st.session_state['video_log'] = df

In [71]:
st.set_page_config(page_title='Sign Language Detector', layout='wide')
st.title('Sign Language Detector')

2025-11-19 17:37:19.387 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 17:37:19.389 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 17:37:19.389 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 17:37:19.390 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [72]:
col1, col2 = st.columns([1, 2])


with col1:
    st.header('Settings')
    st.write(f"Operation window: {OPERATION_START} to {OPERATION_END} ({TIMEZONE})")
    tz = pytz.timezone(TIMEZONE)
    now = datetime.now(tz)
    st.write('Local time: ' + now.strftime('%Y-%m-%d %H:%M:%S'))
    operational = in_operation_hours(now)
    if operational:
        st.success('Within operation hours — predictions enabled')
    else:
        st.warning('Outside operation hours — predictions disabled')


    st.markdown('---')
    st.header('Model')
    model_file = st.text_input('Model file path', MODEL_PATH)
    model = None
    if os.path.exists(model_file):
        try:
            model = load_model(model_file)
            st.success(f'Model loaded from {model_file}')
        except Exception as e:
            st.error(f'Failed to load model: {e}')
    else:
        st.info('Model file not found. Predictions will be disabled until a model is provided.')


    st.markdown('---')
    st.header('Logs')
    if st.button('Download image log (.csv)'):
        csv = st.session_state['image_log'].to_csv(index=False)
        st.download_button('Click to download image log', csv, file_name='image_log.csv')
    if st.button('Download video log (.csv)'):
        csv = st.session_state['video_log'].to_csv(index=False)
        st.download_button('Click to download video log', csv, file_name='video_log.csv')


with col2:
    st.header('Upload Image for Detection')
    uploaded_file = st.file_uploader('Upload an image', type=['png', 'jpg', 'jpeg'])
    if uploaded_file is not None:
        file_bytes = np.asarray(bytearray(uploaded_file.read()), dtype=np.uint8)
        img = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
        st.image(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), caption='Uploaded image', use_column_width=True)
        if not operational:
            st.warning('Outside operation hours: image prediction skipped')
        elif model is None:
            st.error('No model loaded: cannot predict')
        else:
            pred_label, pred_probs = predict_frame_model(model, img)
            if pred_label is None:
                st.info('No hand detected in the image')
            else:
                conf = float(np.max(pred_probs))
                st.success(f'Predicted: {pred_label} (conf {conf:.2f})')
                log_image_detection(pred_label, conf, uploaded_file.name)


    st.markdown('---')
    st.header('Real-time Webcam (Press Start to begin)')
    run_webcam = st.checkbox('Start webcam')
    confidence_threshold = st.slider('Confidence threshold', 0.0, 1.0, 0.6)

2025-11-19 17:37:22.140 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 17:37:22.142 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 17:37:22.144 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 17:37:22.147 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 17:37:22.148 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 17:37:22.149 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 17:37:22.150 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 17:37:22.151 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [73]:
if run_webcam:
    # Initialize webcam capture
    cap = cv2.VideoCapture(0) # 0 for default webcam
    if not cap.isOpened():
        st.error('Could not open webcam. Please ensure it is connected and not in use.')
        run_webcam = False
    else:
        st.warning('Webcam started. Please grant browser permissions if prompted.')

    FRAME_WINDOW = st.image([]) # Placeholder for webcam feed
    frame_idx = 0
    seq_buffer = [] # Buffer for sequence model input

    with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
        while run_webcam and cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                st.error('Failed to grab frame from webcam.')
                break

            # Flip image for natural selfie-view
            frame = cv2.flip(frame, 1)
            image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(image_rgb)

            label_to_show = 'No Hand'
            conf_to_show = None

            if results.multi_hand_landmarks:
                hand = results.multi_hand_landmarks[0]
                fv = hands_to_feature_vector(hand)
                seq_buffer.append(fv)
                if len(seq_buffer) > SEQUENCE_LENGTH:
                    seq_buffer.pop(0)

                # If we have a full sequence, predict using sequence model if available
                if len(seq_buffer) == SEQUENCE_LENGTH:
                    try:
                        # Expect a sequence model; if user loaded a frame model, this will still try to use it and may error
                        seq_in = np.array(seq_buffer).reshape(1, SEQUENCE_LENGTH, 63)
                        pred = model.predict(seq_in)
                        label_idx = np.argmax(pred)
                        label_to_show = LABELS[label_idx]
                        conf_to_show = float(np.max(pred[0]))
                    except Exception:
                        # fallback to frame-level prediction
                        try:
                            pred = model.predict(fv.reshape(1, -1))
                            label_idx = np.argmax(pred)
                            label_to_show = LABELS[label_idx]
                            conf_to_show = float(np.max(pred[0]))
                        except Exception:
                            label_to_show = None

                # Draw hand landmarks
                if results.multi_hand_landmarks:
                    for hl in results.multi_hand_landmarks:
                        mp_drawing.draw_landmarks(frame, hl, mp_hands.HAND_CONNECTIONS)


            # Overlay prediction
            if label_to_show and conf_to_show is not None and conf_to_show >= confidence_threshold:
                cv2.putText(frame, f'{label_to_show} (conf {conf_to_show:.2f})', (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                log_video_detection(label_to_show, conf_to_show, frame_idx)

            FRAME_WINDOW.image(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            frame_idx += 1

            # allow Streamlit to update UI; check whether the user unchecked the box
            run_webcam = st.checkbox('Start webcam', value=True)

    cap.release()
    cv2.destroyAllWindows()


# -----------------------------
# Training helpers (offline; run separately)
# -----------------------------
st.markdown('---')
st.header('Training helpers (for dataset authors)')
st.write('This section outlines how to collect data and train a model. You should run training offline (not in Streamlit live server).')

st.subheader('Data collection (recommended)')
st.write('1. For frame-level model: capture single images of hands doing signs.\n2. For sequence model: record short video clips (e.g., 30 frames) for each sign.\n3. Use MediaPipe to extract and store the 63-dim landmark vectors per frame. Save as numpy arrays and label them.')

st.subheader('Simple training example (frame-level)')
st.code('''
# Pseudocode / example (run in a separate script)
# X: numpy array shape (n_samples, 63)
# y: list/array of labels (n_samples,)
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_enc = le.fit_transform(y)
y_cat = to_categorical(y_enc)

model = build_frame_model(num_classes=len(le.classes_))
model.fit(X, y_cat, epochs=50, batch_size=32, validation_split=0.2)
model.save('sign_model.h5')
''')

2025-11-19 17:37:27.109 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 17:37:27.110 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 17:37:27.112 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 17:37:27.114 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 17:37:27.117 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 17:37:27.118 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 17:37:27.119 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 17:37:27.122 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()

In [74]:
st.subheader('Simple training example (sequence-level)')
st.code('''
# X_seq: numpy array shape (n_clips, seq_len, 63)
# y_seq: labels shape (n_clips,)
le = LabelEncoder()
y_enc = le.fit_transform(y_seq)
y_cat = to_categorical(y_enc)
model = build_sequence_model(num_classes=len(le.classes_), seq_len=X_seq.shape[1])
model.fit(X_seq, y_cat, epochs=80, batch_size=8, validation_split=0.2)
model.save('sign_sequence_model.h5')
''')


st.markdown('---')
st.write('If you want, I can provide a separate training script that walks through dataset creation, augmentation, and model tuning.')

2025-11-19 17:37:31.202 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 17:37:31.203 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 17:37:31.207 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 17:37:31.208 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 17:37:31.209 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 17:37:31.211 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 17:37:31.215 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 17:37:31.215 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar